In [3]:
pip install transformers

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [5]:
file_path=pd.read_csv(r'C:\Users\rohit\OneDrive\Desktop\pro\final.csv')


In [6]:
file_path.head()

,Unnamed: 0,quastion,answer,label
0,0,what composer used sound mass,"['Composers and works include Barbara Kolb , P...",human_ans
1,1,what composer used sound mass,['Sound mass is a term used to describe a type...,chat_gpt
2,2,where did the persian war take place,['The Greco-Persian Wars (also often called th...,human_ans
3,3,where did the persian war take place,['The Persian Wars were a series of conflicts ...,chat_gpt
4,4,what are add ons,"['Plug-in (computing) , a piece of software wh...",human_ans


In [7]:
file_path.label=='human_ans'

file_path['label'] = np.where(file_path['label']=='chat_gpt', '1', '0')

file_path =file_path.rename({'answer': 'text'}, axis='columns')

file_path.head()

,Unnamed: 0,quastion,text,label
0,0,what composer used sound mass,"['Composers and works include Barbara Kolb , P...",0
1,1,what composer used sound mass,['Sound mass is a term used to describe a type...,1
2,2,where did the persian war take place,['The Greco-Persian Wars (also often called th...,0
3,3,where did the persian war take place,['The Persian Wars were a series of conflicts ...,1
4,4,what are add ons,"['Plug-in (computing) , a piece of software wh...",0


In [8]:
file_path[["label"]] = file_path[["label"]].apply(pd.to_numeric)


file_path.info()

file_path.label

text = file_path.text.values
labels = file_path.label.values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6554 entries, 0 to 6553
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6554 non-null   int64 
 1   quastion    6554 non-null   object
 2   text        6554 non-null   object
 3   label       6554 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 204.9+ KB


In [9]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )

In [10]:
def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0,len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒═════════════════╤═════════════╕
│ Tokens          │   Token IDs │
╞═════════════════╪═════════════╡
│ [               │        1031 │
├─────────────────┼─────────────┤
│ "               │        1000 │
├─────────────────┼─────────────┤
│ in              │        1999 │
├─────────────────┼─────────────┤
│ science         │        2671 │
├─────────────────┼─────────────┤
│ ,               │        1010 │
├─────────────────┼─────────────┤
│ computing       │        9798 │
├─────────────────┼─────────────┤
│ ,               │        1010 │
├─────────────────┼─────────────┤
│ and             │        1998 │
├─────────────────┼─────────────┤
│ engineering     │        3330 │
├─────────────────┼─────────────┤
│ ,               │        1010 │
├─────────────────┼─────────────┤
│ a               │        1037 │
├─────────────────┼─────────────┤
│ black           │        2304 │
├─────────────────┼─────────────┤
│ box             │        3482 │
├─────────────────┼─────────────┤
│ is          

In [11]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [12]:
file_path.head()

,Unnamed: 0,quastion,text,label
0,0,what composer used sound mass,"['Composers and works include Barbara Kolb , P...",0
1,1,what composer used sound mass,['Sound mass is a term used to describe a type...,1
2,2,where did the persian war take place,['The Greco-Persian Wars (also often called th...,0
3,3,where did the persian war take place,['The Persian Wars were a series of conflicts ...,1
4,4,what are add ons,"['Plug-in (computing) , a piece of software wh...",0


In [13]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [14]:
for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])

token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\rohit\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [22]:
token_id[50]

tensor([  101,  1031,  1000,  2304,  5958,  2003,  1996,  2154,  2206, 15060,
         2154,  1999,  1996,  2142,  2163,  1010,  2411,  5240,  2004,  1996,
         2927,  1997,  1996,  4234,  6023,  2161,  1012,  2023,  2001,  2579,
         2000,  1037,  2047,  6034,  1999,  2249,  1010,  2043,  2195, 16629,
         1006,  2164,  4539,  1010, 12849,  7317,  1005,  1055,  1010, 20914,
         1005,  1055,  1010,  2190,  4965,  1010,  1998, 26892, 12718,  1007,
         2441,  2012,  7090,   102])

In [40]:
def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text)+1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence_encoding()

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │         101 │                1 │
├──────────┼─────────────┼──────────────────┤
│ [        │        1031 │                1 │
├──────────┼─────────────┼──────────────────┤
│ '        │        1005 │                1 │
├──────────┼─────────────┼──────────────────┤
│ fruit    │        5909 │                1 │
├──────────┼─────────────┼──────────────────┤
│ cake     │        9850 │                1 │
├──────────┼─────────────┼──────────────────┤
│ (        │        1006 │                1 │
├──────────┼─────────────┼──────────────────┤
│ or       │        2030 │                1 │
├──────────┼─────────────┼──────────────────┤
│ fruit    │        5909 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##cake   │       17955 │                1 │
├──────────┼─────────────┼──────────────────┤
│ )        │        1007 │        

In [43]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 64

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(0,len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [44]:
file_path.head()

,Unnamed: 0,quastion,text,label
0,0,what composer used sound mass,"['Composers and works include Barbara Kolb , P...",0
1,1,what composer used sound mass,['Sound mass is a term used to describe a type...,1
2,2,where did the persian war take place,['The Greco-Persian Wars (also often called th...,0
3,3,where did the persian war take place,['The Persian Wars were a series of conflicts ...,1
4,4,what are add ons,"['Plug-in (computing) , a piece of software wh...",0


In [45]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [47]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
#model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  50%|█████     | 1/2 [43:08<43:08, 2588.36s/it]


	 - Train loss: 0.2217
	 - Validation Accuracy: 0.9717
	 - Validation Precision: 0.9598
	 - Validation Recall: 0.9852
	 - Validation Specificity: 0.9579



Epoch: 100%|██████████| 2/2 [2:41:57<00:00, 4858.88s/it]


	 - Train loss: 0.0715
	 - Validation Accuracy: 0.9627
	 - Validation Precision: 0.9370
	 - Validation Recall: 0.9911
	 - Validation Specificity: 0.9346



In [60]:
new_sentence='Hi..The pain you feel in the neck is due to muscle spasm...the spasm could occur due to irritation of neural structures or facet joints...In your case posture plays a major role in contributing to the pain...I suggest you toPut lot if icing frequently to the back of neck...A rolled towel under your neck curvature...will help you relax well...The more you hold your neck in proper posture the less pain you will have... You can modify your work station to maintain the posture...Try gentle stretches like bending your neck to touch your chin to chest...Bend your neck sideways till you feel stretch and not pain.. hold it in that stage...look up to the ceiling and push chin upwards...Once pain comes down I would like you to start your training for upper back...neck...like retraction work for your upper back..and pushing your neck with your hands clasped and neck pushing back...happening simultaneously....Hope this information help you to get better...Revert back incase you need further clarification..'
Actual_label='human_ans'
# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

prediction = 'chat_gpt' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'human_ans'

print('Input Sentence: ', new_sentence)
print(Actual_label)
print('Predicted Class: ',Actual_label,'=', prediction)

Input Sentence:  Hi..The pain you feel in the neck is due to muscle spasm...the spasm could occur due to irritation of neural structures or facet joints...In your case posture plays a major role in contributing to the pain...I suggest you toPut lot if icing frequently to the back of neck...A rolled towel under your neck curvature...will help you relax well...The more you hold your neck in proper posture the less pain you will have... You can modify your work station to maintain the posture...Try gentle stretches like bending your neck to touch your chin to chest...Bend your neck sideways till you feel stretch and not pain.. hold it in that stage...look up to the ceiling and push chin upwards...Once pain comes down I would like you to start your training for upper back...neck...like retraction work for your upper back..and pushing your neck with your hands clasped and neck pushing back...happening simultaneously....Hope this information help you to get better...Revert back incase you ne

c:\Users\rohit\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
